In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

## 1. Load Data

In [2]:
df_listing1 = pd.read_csv('berlin-airbnb-data/listings_summary.csv')
review = pd.read_csv('review_clean.csv')

In [3]:
pd.set_option('max_columns', None)
df_listing1.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2015,https://www.airbnb.com/rooms/2015,20181107122246,2018-11-07,Berlin-Mitte Value! Quiet courtyard/very central,Great location! 30 of 75 sq meters. This wood...,A+++ location! This „Einliegerwohnung“ is an e...,Great location! 30 of 75 sq meters. This wood...,none,It is located in the former East Berlin area o...,"This is my home, not a hotel. I rent out occas...","Close to U-Bahn U8 and U2 (metro), Trams M12, ...","Simple kitchen/cooking, refrigerator, microwav...",Always available,"No parties No events No pets No smoking, not e...",NaN,NaN,https://a0.muscache.com/im/pictures/260fd609-7...,NaN,2217,https://www.airbnb.com/users/show/2217,Ian,2008-08-18,"Key Biscayne, Florida, United States",Believe in sharing economy.,within an hour,96%,NaN,t,https://a0.muscache.com/im/pictures/21428a22-4...,https://a0.muscache.com/im/pictures/21428a22-4...,Mitte,4.0,4.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,"Berlin, Berlin, Germany",Mitte,Brunnenstr. Süd,Mitte,Berlin,Berlin,10119,Berlin,"Berlin, Germany",DE,Germany,52.534537,13.402557,f,Guesthouse,Entire home/apt,3,1.0,1.0,2.0,Real Bed,"{TV,""Cable TV"",Wifi,Kitchen,Gym,Heating,""Famil...",NaN,$60.00,NaN,NaN,$200.00,$30.00,1,$28.00,4,1125,3 months ago,t,0,21,51,141,2018-11-07,118,2016-04-11,2018-10-28,93.0,10.0,9.0,10.0,10.0,10.0,9.0,t,NaN,NaN,f,f,strict_14_with_grace_period,f,f,4,3.76
1,2695,https://www.airbnb.com/rooms/2695,20181107122246,2018-11-07,Prenzlauer Berg close to Mauerpark,NaN,In the summertime we are spending most of our ...,In the summertime we are spending most of our ...,none,NaN,NaN,Within walking distance you'll find the S-Bahn...,Außer deinem Zimmer kannst du noch die Küche u...,NaN,NaN,NaN,NaN,https://a0.muscache.com/im/pictures/97d4f6e8-1...,NaN,2986,https://www.airbnb.com/users/show/2986,Michael,2008-09-16,"Berlin, Berlin, Germany",Living with my wife in Berlin. Like travellin...,NaN,NaN,NaN,f,https://a0.muscache.com/im/pictures/50434955-1...,https://a0.muscache.com/im/pictures/50434955-1...,Prenzlauer Berg,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,"Berlin, Berlin, Germany",NaN,Prenzlauer Berg Nordwest,Pankow,Berlin,Berlin,10437,Berlin,"Berlin, Germany",DE,Germany,52.548513,13.404553,t,Apartment,Private room,2,1.0,1.0,1.0,Real Bed,"{Wifi,Kitchen,Elevator,Heating,Washer,Essentia...",NaN,$17.00,NaN,NaN,$0.00,$0.00,1,$0.00,2,40,7 weeks ago,t,0,0,0,0,2018-11-07,6,2018-07-04,2018-10-01,100.0,10.0,10.0,10.0,10.0,10.0,10.0,t,NaN,NaN,f,f,flexible,f,f,1,1.42
2,3176,https://www.airbnb.com/rooms/3176,20181107122246,2018-1

Some of the more important features this project will look into are the following: 

- accommodates : the number of guests the rental can accommodate
- bedrooms : number of bedrooms included in the rental
- bathrooms : number of bathrooms included in the rental
- beds : number of beds included in the rental
- price: nightly price for the rental
- minimum_nights : minimum number of nights a guest can stay for the rental
- maximum_nights: maximum number of nights a guest can stay for the rental
- number_of_reviews: number of reviews that previous guests have left



In [4]:
df_listing1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22552 entries, 0 to 22551
Data columns (total 96 columns):
id                                  22552 non-null int64
listing_url                         22552 non-null object
scrape_id                           22552 non-null int64
last_scraped                        22552 non-null object
name                                22493 non-null object
summary                             21589 non-null object
space                               14020 non-null object
description                         22349 non-null object
experiences_offered                 22552 non-null object
neighborhood_overview               11540 non-null object
notes                               7215 non-null object
transit                             13036 non-null object
access                              10837 non-null object
interaction                         10406 non-null object
house_rules                         11449 non-null object
thumbnail_url           

The feature will be use in this project:
- neighbourhood_cleansed - the berlin the property is in
- property_type - type of property, e.g. house or flat
- room_type - type of listing, e.g. entire home, private room or shared room
- accommodates - how many people the property accommodates
- bathrooms - number of bathrooms
- bedrooms - number of bedrooms
- beds - number of beds
- bed_type - type of bed, e.g. real bed or sofa-bed
- amenities - list of amenities
- price - nightly advertised price (the target variable)
- security_deposit - the amount required as a security deposit
- cleaning_fee - the amount of the cleaning fee
- guests_included - the number of guests included in the booking fee
- extra_people - the price per additional guest above the guests_included price
- minimum_nights - the minimum length of stay
- maximum_nights - the maximum length of stay
- number_of_reviews - the number of reviews left for the property
- first_review - the date of the first review
- last_review - the date of the most recent review
- review_scores_rating - guests can score properties overall from 1 to 5 stars
- instant_bookable - whether or not the property can be instant booked (i.e. booked straight away, without having to message the host first and wait to be accepted)
- cancellation_policy - the type of cancellation policy, e.g. strict or moderate
- reviews_per_month - calculated field of the average number of reviews left by guest each month

In [5]:
columns_use = ['neighbourhood_cleansed',
                   'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',  
                   'bedrooms', 'bed_type','amenities', 'price', 'cleaning_fee', 
                   'security_deposit', 'extra_people', 'guests_included', 'minimum_nights','maximum_nights',
                   'review_scores_rating','instant_bookable', 'cancellation_policy']

df_list = df_listing1[columns_use]

In [6]:
df_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22552 entries, 0 to 22551
Data columns (total 20 columns):
neighbourhood_cleansed    22552 non-null object
latitude                  22552 non-null float64
longitude                 22552 non-null float64
property_type             22552 non-null object
room_type                 22552 non-null object
accommodates              22552 non-null int64
bathrooms                 22520 non-null float64
bedrooms                  22534 non-null float64
bed_type                  22552 non-null object
amenities                 22552 non-null object
price                     22552 non-null object
cleaning_fee              15406 non-null object
security_deposit          13191 non-null object
extra_people              22552 non-null object
guests_included           22552 non-null int64
minimum_nights            22552 non-null int64
maximum_nights            22552 non-null int64
review_scores_rating      18163 non-null float64
instant_bookable          

In [7]:
listItem = []
for col in df_list.columns :
    listItem.append([col, df_list[col].dtype, df_list[col].isna().sum(), round((df_list[col].isna().sum()/len(df_list[col])) * 100,2),
                    df_list[col].nunique(), df_list[col].drop_duplicates().sample(2, replace=True).values])

dfDesc = pd.DataFrame(columns=['dataFeatures', 'dataType', 'null', 'nullPct', 'unique', 'uniqueSample'],
                     data=listItem)
dfDesc

,dataFeatures,dataType,null,nullPct,unique,uniqueSample
0,neighbourhood_cleansed,object,0,0.00,136,"[Tempelhof, Prenzlauer Berg Süd]"
1,latitude,float64,0,0.00,22552,"[52.512167434704246, 52.4682666530344]"
2,longitude,float64,0,0.00,22552,"[13.329413880801887, 13.396690435426267]"
3,property_type,object,0,0.00,33,"[Houseboat, Boutique hotel]"
4,room_type,object,0,0.00,3,"[Shared room, Private room]"
5,accommodates,int64,0,0.00,16,"[5, 15]"
6,bathrooms,float64,32,0.14,16,"[1.5, 5.0]"
7,bedrooms,float64,18,0.08,12,"[6.0, 2.0]"
8,bed_type,object,0,0.00,5,"[Couch, Futon]"
9,amenities,object,0,0.00,19985,"[{Wifi,Kitchen,Heating,Washer,Dryer,Essentials..."


## 2. Data Cleaning Listing Dataset

### 2.1 Price Column

- Formatting the price

In [8]:
df_list.price = df_list.price.str.replace('$','')
df_list.price = df_list.price.str.replace(',','').astype(np.float64)
df_list.price = round(df_list.price, 2)

In [9]:
print(f"Prices range from ${min(df_list.price)} to ${max(df_list.price)}.")

Prices range from $0.0 to $9000.0.


In [10]:
# Delete price $0
df_list.drop(df_list[df_list['price'] == 0].index, inplace=True)
df_list.reset_index(drop=True, inplace=True)

In [11]:
print(f"Prices range from ${min(df_list.price)} to ${max(df_list.price)}.")

Prices range from $1.0 to $9000.0.


In [12]:
# Delete price $1
df_list.drop(df_list[df_list['price'] == 1].index, inplace=True)
df_list.reset_index(drop=True, inplace=True)

In [13]:
print(f"Prices range from ${min(df_list.price)} to ${max(df_list.price)}.")

Prices range from $8.0 to $9000.0.


### 2.2 Bathrooms Column

- Check if bathrooms Have NaN values

In [14]:
print('Sum of Null in Bathrooms Column is {}'.format(df_list.bathrooms.isnull().sum()))

Sum of Null in Bathrooms Column is 32


In [15]:
# Fill NaN with 0
df_list.bathrooms.fillna(0, inplace=True)

> So why do I fill NaN with 0? because I assume that this property does not have a bathroom

In [16]:
pd.set_option('max_rows', None)
df_list[df_list['bathrooms'] < 1]

,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,bed_type,amenities,price,cleaning_fee,security_deposit,extra_people,guests_included,minimum_nights,maximum_nights,review_scores_rating,instant_bookable,cancellation_policy
21,Reuterstraße,52.486106,13.434817,Apartment,Entire home/apt,3,0.0,1.0,Real Bed,{},60.0,$30.00,NaN,$10.00,2,2,730,100.0,f,flexible
27,Otto-Suhr-Allee,52.519677,13.302089,Apartment,Private room,1,0.0,1.0,Real Bed,"{Heating,Essentials,Hangers,""Hair dryer"",Iron,...",40.0,$5.00,$500.00,$20.00,1,1,21,88.0,f,flexible
47,Schöneberg-Nord,52.498379,13.341816,Apartment,Shared room,1,0.0,1.0,Couch,{},88.0,$10.00,NaN,$0.00,1,2,3,NaN,f,moderate
93,Tempelhofer Vorstadt,52.490975,13.376910,Apartment,Entire home/apt,4,0.0,2.0,Real Bed,"{Internet,Wifi,Kitchen,""Buzzer/wireless interc...",75.0,$40.00,NaN,$10.00,2,2,60,95.0,f,moderate
148,Brunnenstr. Süd,52.530464,13.398893,Apartment,Entire home/apt,2,0.0,1.0,Futon,"{Internet,Wifi,Kitchen,""Buzzer/wireless interc...",80.0,NaN,$100.00,$100.00,2,4,365,97.0,f,moderate
180,Otto-Suhr-Allee,52.519934,13.300558,Apartment,Private room,1,0.0,1.0,Real Bed,"{Heating,Essentials,Hangers,""Hair dryer"",Iron,...",51.0,$5.00,$500.00,$20.00,1,1,27,83.0,t,flexible
184,Ostpreußendamm,52.430104,13.326931,Apartment,Private room,2,0.5,1.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Pets allowed"",""Free...",45.0,$20.00,NaN,$20.00,1,2,15,96.0,f,moderate
231,West 2,52.584223,13.231135,Apartment,Entire home/apt,2,0.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Pets all...",23.0,NaN,$650.00,$0.00,2,30,365,100.0,t,strict_14_with_grace_period
232,West 2,52.582027,13.230722,Apartment,Entire home/apt,2,0.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Pets all...",19.0,NaN,$550.00,$0.00,2,30,365,83.0,t,strict_14_with_grace_period
274,Schöneberg-Nord,52.499264,13.339909,Apartment,Entire home/apt,2,0.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Kitchen,""Washer / Drye...",69.0,$20.00,$300.00,$0.00,1,4,90,NaN,f,flexible


> We can see the list above that there is a bathroom 0.5 according to the real estate listing it is a powder room that only has Westafle and toilets do not have a shower or bathtub and in this list there is also a 1.5 bathroom which means that this bathroom is shared with the owner property or other guests

In [17]:
print('Number of Null in Bathrooms Column is {}'.format(df_list.bathrooms.isnull().sum()))

Number of Null in Bathrooms Column is 0


### 2.3 Bedrooms Column

- Check if bedrooms have NaN

In [18]:
print('Number of Null in Bedrooms Column is {}'.format(df_list.bedrooms.isnull().sum()))

Number of Null in Bedrooms Column is 18


In [19]:
# Fill NaN with 0
df_list.bedrooms.fillna(0, inplace=True)

> So why did I fill NaN with 0? because according to U.S. Department of Housing & Urban Development
Office of Policy Development & Research: A room is built as a bedroom, although not used for that purpose, such as a
room meant to be a bedroom but used as a sewing room, is counted as a bedroom. On the
other hand, a room designed and used mainly for other purposes, such as a den with a
sleep sofa used mainly for watching television, is not considered a bedroom. **A housing
The unit consists of only one room, such as a one-room efficiency apartment, is classified
by definition as having no bedroom**. https://www.huduser.gov/portal/publications/AHS_NumberOfBedrooms.pdf

### 2.4 Cleaning Fee, Security Deposit & Extra People Column

- Formatting the Value
- Fill missing value with 0 because i assume that is free

In [20]:
print('Number of Null in Cleaning fee Column is {}'.format(df_list.cleaning_fee.isnull().sum()))
print('Number of Null in Security Deposit Column is {}'.format(df_list.security_deposit.isnull().sum()))
print('Number of Null in Extra People Column is {}'.format(df_list.extra_people.isnull().sum()))

Number of Null in Cleaning fee Column is 7146
Number of Null in Security Deposit Column is 9360
Number of Null in Extra People Column is 0


In [21]:
df_list.cleaning_fee = df_list.cleaning_fee.str.replace('$','')
df_list.cleaning_fee = df_list.cleaning_fee.str.replace(',','').astype(np.float64)
df_list.cleaning_fee.fillna(0, inplace=True)
df_list.security_deposit = df_list.security_deposit.str.replace('$','')
df_list.security_deposit = df_list.security_deposit.str.replace(',','').astype(np.float64)
df_list.security_deposit.fillna(0, inplace=True)
df_list.extra_people = df_list.extra_people.str.replace('$','')
df_list.extra_people = df_list.extra_people.str.replace(',','').astype(np.float64)

### 2.5 Amenities Column

In [22]:
df_list.head()

,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,bed_type,amenities,price,cleaning_fee,security_deposit,extra_people,guests_included,minimum_nights,maximum_nights,review_scores_rating,instant_bookable,cancellation_policy
0,Brunnenstr. Süd,52.534537,13.402557,Guesthouse,Entire home/apt,3,1.0,1.0,Real Bed,"{TV,""Cable TV"",Wifi,Kitchen,Gym,Heating,""Famil...",60.0,30.0,200.0,28.0,1,4,1125,93.0,f,strict_14_with_grace_period
1,Prenzlauer Berg Nordwest,52.548513,13.404553,Apartment,Private room,2,1.0,1.0,Real Bed,"{Wifi,Kitchen,Elevator,Heating,Washer,Essentia...",17.0,0.0,0.0,0.0,1,2,40,100.0,f,flexible
2,Prenzlauer Berg Südwest,52.534996,13.417579,Apartment,Entire home/apt,4,1.0,1.0,Real Bed,"{Internet,Wifi,Kitchen,""Buzzer/wireless interc...",90.0,50.0,200.0,20.0,2,62,1125,92.0,t,strict_14_with_grace_period
3,Schöneberg-Nord,52.498855,13.349065,Apartment,Private room,2,1.0,1.0,Pull-out Sofa,"{Internet,Wifi,""Pets allowed"",""Pets live on th...",26.0,30.0,250.0,18.0,1,5,360,88.0,f,strict_14_with_grace_period
4,Helmholtzplatz,52.543157,13.415091,Apartment,Private room,2,1.0,1.0,Real Bed,"{Wifi,Heating,""Family/kid friendly"",Essentials...",42.0,0.0,0.0,24.0,1,2,10,96.0,f,moderate


In [23]:
# Example of amenities listed
df_list.amenities[:1].values

array(['{TV,"Cable TV",Wifi,Kitchen,Gym,Heating,"Family/kid friendly","Smoke detector",Essentials,Shampoo,"Lock on bedroom door",Hangers,"Hair dryer",Iron,"Laptop friendly workspace","Private living room",Bathtub,"Hot water","Bed linens","Extra pillows and blankets",Microwave,"Coffee maker",Refrigerator,Dishwasher,"Dishes and silverware","Cooking basics",Stove,"Luggage dropoff allowed","Long term stays allowed"}'],
      dtype=object)

In [24]:
### Creating a set of all possible amenities
amenities_list = list(df_list.amenities)
amenities_list_string = " ".join(amenities_list)
amenities_list_string = amenities_list_string.replace('{', '')
amenities_list_string = amenities_list_string.replace('}', ',')
amenities_list_string = amenities_list_string.replace('"', '')
amenities_set = [x.strip() for x in amenities_list_string.split(',')]
amenities_set = set(amenities_set)
amenities_set

{'',
 '24-hour check-in',
 'Accessible-height bed',
 'Accessible-height toilet',
 'Air conditioning',
 'Air purifier',
 'BBQ grill',
 'Baby bath',
 'Baby monitor',
 'Babysitter recommendations',
 'Bath towel',
 'Bathroom essentials',
 'Bathtub',
 'Bathtub with bath chair',
 'Beach essentials',
 'Beachfront',
 'Bed linens',
 'Bedroom comforts',
 'Body soap',
 'Breakfast',
 'Breakfast table',
 'Building staff',
 'Buzzer/wireless intercom',
 'Cable TV',
 'Carbon monoxide detector',
 'Cat(s)',
 'Ceiling hoist',
 'Changing table',
 'Children’s books and toys',
 'Children’s dinnerware',
 'Cleaning before checkout',
 'Coffee maker',
 'Convection oven',
 'Cooking basics',
 'Crib',
 'Disabled parking spot',
 'Dishes and silverware',
 'Dishwasher',
 'Dog(s)',
 'Doorman',
 'Dryer',
 'EV charger',
 'Electric profiling bed',
 'Elevator',
 'Essentials',
 'Ethernet connection',
 'Extra pillows and blankets',
 'Family/kid friendly',
 'Fire extinguisher',
 'Fireplace guards',
 'Firm mattress',
 'First 

In the list above, we can see many facilities that actually do not affect the price increase, for example 'Hair dryer', so in this case I choose a facility that can increase property prices.

The amenities chosen are (slashes indicate separate categories that can be combined):


- Air conditioning/central air conditioning
- Amazon Echo/Apple TV/DVD player/game console/Netflix/projector and screen/smart TV (i.e. non-basic electronics)
- Balcony/patio or balcony
- Beach view/beachfront/lake access/mountain view/ski-in ski-out/waterfront (i.e. great location/views)
- Bed linens
- Breakfast
- Cable TV/TV
- Coffee maker/espresso machine
- Cooking basics
- Dishwasher/Dryer/Washer/Washer and dryer
- Elevator
- Exercise equipment/gym/private gym/shared gym
- Family/kid friendly, or anything containing 'children'
- Free parking on premises/free street parking/outdoor parking/paid parking off premises/paid parking on premises
- Garden or backyard/outdoor seating/sun loungers/terrace
- Hot tub/jetted tub/private hot tub/sauna/shared hot tub/pool/private pool/shared pool
- Internet/pocket wifi/wifi
- Long term stays allowed
- Pets allowed/cat(s)/dog(s)/pets live on this property/other pet(s)
- Safe/security system
- Smoking allowed
- Step-free access/wheelchair accessible, or anything containing 'accessible'
- Suitable for events

In [25]:
df_list.loc[df_list['amenities'].str.contains('Air conditioning|Central air conditioning'), 'air_conditioning'] = 1
df_list.loc[df_list['amenities'].str.contains('Amazon Echo|Apple TV|Game console|Netflix|Projector and screen|Smart TV'), 'high_end_electronics'] = 1
df_list.loc[df_list['amenities'].str.contains('Balcony|Patio'), 'balcony'] = 1
df_list.loc[df_list['amenities'].str.contains('Beach view|Beachfront|Lake access|Mountain view|Ski-in/Ski-out|Waterfront'), 'nature_and_views'] = 1
df_list.loc[df_list['amenities'].str.contains('Bed linens'), 'bed_linen'] = 1
df_list.loc[df_list['amenities'].str.contains('Breakfast'), 'breakfast'] = 1
df_list.loc[df_list['amenities'].str.contains('TV'), 'tv'] = 1
df_list.loc[df_list['amenities'].str.contains('Coffee maker|Espresso machine'), 'coffee_machine'] = 1
df_list.loc[df_list['amenities'].str.contains('Cooking basics'), 'cooking_basics'] = 1
df_list.loc[df_list['amenities'].str.contains('Dishwasher|Dryer|Washer'), 'dishwasher'] = 1
df_list.loc[df_list['amenities'].str.contains('Elevator'), 'elevator'] = 1
df_list.loc[df_list['amenities'].str.contains('Exercise equipment|Gym|gym'), 'gym'] = 1
df_list.loc[df_list['amenities'].str.contains('Family/kid friendly|Children|children'), 'child_friendly'] = 1
df_list.loc[df_list['amenities'].str.contains('parking'), 'parking'] = 1
df_list.loc[df_list['amenities'].str.contains('Garden|Outdoor|Sun loungers|Terrace'), 'outdoor_space'] = 1
df_list.loc[df_list['amenities'].str.contains('Hot tub|Jetted tub|hot tub|Sauna|Pool|pool'), 'hot_tub_sauna_or_pool'] = 1
df_list.loc[df_list['amenities'].str.contains('Internet|Pocket wifi|Wifi'), 'internet'] = 1
df_list.loc[df_list['amenities'].str.contains('Long term stays allowed'), 'long_term_stays'] = 1
df_list.loc[df_list['amenities'].str.contains('Pets|pet|Cat(s)|Dog(s)'), 'pets_allowed'] = 1
df_list.loc[df_list['amenities'].str.contains('Safe|Security system'), 'secure'] = 1
df_list.loc[df_list['amenities'].str.contains('Smoking allowed'), 'smoking_allowed'] = 1
df_list.loc[df_list['amenities'].str.contains('Step-free access|Wheelchair|Accessible'), 'accessible'] = 1
df_list.loc[df_list['amenities'].str.contains('Suitable for events'), 'event_suitable'] = 1

In [26]:
df_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 0 to 22542
Data columns (total 43 columns):
neighbourhood_cleansed    22543 non-null object
latitude                  22543 non-null float64
longitude                 22543 non-null float64
property_type             22543 non-null object
room_type                 22543 non-null object
accommodates              22543 non-null int64
bathrooms                 22543 non-null float64
bedrooms                  22543 non-null float64
bed_type                  22543 non-null object
amenities                 22543 non-null object
price                     22543 non-null float64
cleaning_fee              22543 non-null float64
security_deposit          22543 non-null float64
extra_people              22543 non-null float64
guests_included           22543 non-null int64
minimum_nights            22543 non-null int64
maximum_nights            22543 non-null int64
review_scores_rating      18156 non-null float64
instant_bookable      

One way to reduce the number of features is to remove the amenities which add relatively little information, or are relatively unhelpful in differentiating between different listings. Amenity features where either the true or the false category contains fewer than 10% of listings will be removed.

In [27]:
# Replacing nulls with zeros for new columns
cols_to_replace_nulls = df_list.iloc[:,20:].columns
df_list[cols_to_replace_nulls] = df_list[cols_to_replace_nulls].fillna(0)

infrequent_amenities = []
for col in df_list.iloc[:,20:].columns:
    if df_list[col].sum() < len(df_list)/10:
        infrequent_amenities.append(col)
print(infrequent_amenities)

# Dropping infrequent amenity features
df_list.drop(infrequent_amenities, axis=1, inplace=True)

# Dropping the original amenity feature
df_list.drop('amenities', axis=1, inplace=True)

['air_conditioning', 'high_end_electronics', 'nature_and_views', 'breakfast', 'gym', 'outdoor_space', 'hot_tub_sauna_or_pool', 'secure', 'accessible', 'event_suitable']


In [28]:
df_list.columns[20:]

Index(['bed_linen', 'tv', 'coffee_machine', 'cooking_basics', 'dishwasher',
       'elevator', 'child_friendly', 'parking', 'internet', 'long_term_stays',
       'pets_allowed', 'smoking_allowed'],
      dtype='object')

In [29]:
df_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 0 to 22542
Data columns (total 32 columns):
neighbourhood_cleansed    22543 non-null object
latitude                  22543 non-null float64
longitude                 22543 non-null float64
property_type             22543 non-null object
room_type                 22543 non-null object
accommodates              22543 non-null int64
bathrooms                 22543 non-null float64
bedrooms                  22543 non-null float64
bed_type                  22543 non-null object
price                     22543 non-null float64
cleaning_fee              22543 non-null float64
security_deposit          22543 non-null float64
extra_people              22543 non-null float64
guests_included           22543 non-null int64
minimum_nights            22543 non-null int64
maximum_nights            22543 non-null int64
review_scores_rating      18156 non-null float64
instant_bookable          22543 non-null object
cancellation_policy   

### 2.6 Review Score Rating Column

- Fill NaN value with 0

In [30]:
print('Number of Null in Review Score Column is {}'.format(df_list.review_scores_rating.isnull().sum()))

Number of Null in Review Score Column is 4387


In [31]:
df_list.review_scores_rating.fillna(0, inplace=True)

In [32]:
print('Number of Null in Review Score Column is {}'.format(df_list.review_scores_rating.isnull().sum()))

Number of Null in Review Score Column is 0


### 2.7 Property Type Column

In [33]:
df_list.property_type.value_counts()

Apartment                 20218
Condominium                 612
Loft                        460
House                       397
Serviced apartment          174
Hostel                      128
Townhouse                    99
Guest suite                  74
Bed and breakfast            64
Guesthouse                   57
Hotel                        50
Other                        47
Boutique hotel               43
Bungalow                     20
Boat                         17
Tiny house                   12
Houseboat                    11
Camper/RV                    11
Villa                        10
Pension (South Korea)         7
Aparthotel                    7
Cabin                         6
Cottage                       4
Resort                        3
Train                         2
Castle                        2
Casa particular (Cuba)        2
Cave                          1
In-law                        1
Island                        1
Tipi                          1
Chalet  

> We can see from the list above that there are many types of properties that are actually the same as serviced apartments and apartments, and that there are only a small number of types of properties. so I decided to name others

In [34]:
df_list.property_type.replace({
    'Townhouse': 'House',
    'Serviced apartment': 'Apartment',
    'Loft': 'Apartment',
    'Bungalow': 'House',
    'Cottage': 'House',
    'Villa': 'House',
    'Tiny house': 'House',
    'Chalet': 'House',
    }, inplace=True)
df_list.loc[~df_list.property_type.isin(['House','Apartment']), 'property_type'] = 'Other'

### 2.8 Change type of int to float

In [35]:
df_list.accommodates = df_list.accommodates.astype(np.float64)
df_list.guests_included = df_list.guests_included.astype(np.float64)
df_list.maximum_nights = df_list.maximum_nights.astype(np.float64)
df_list.minimum_nights = df_list.minimum_nights.astype(np.float64)

In [36]:
df_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22543 entries, 0 to 22542
Data columns (total 32 columns):
neighbourhood_cleansed    22543 non-null object
latitude                  22543 non-null float64
longitude                 22543 non-null float64
property_type             22543 non-null object
room_type                 22543 non-null object
accommodates              22543 non-null float64
bathrooms                 22543 non-null float64
bedrooms                  22543 non-null float64
bed_type                  22543 non-null object
price                     22543 non-null float64
cleaning_fee              22543 non-null float64
security_deposit          22543 non-null float64
extra_people              22543 non-null float64
guests_included           22543 non-null float64
minimum_nights            22543 non-null float64
maximum_nights            22543 non-null float64
review_scores_rating      22543 non-null float64
instant_bookable          22543 non-null object
cancellation_p

### 2.9 Cancellation Policy Column

In [42]:
df_list.cancellation_policy.value_counts()

flexible                       9101
moderate                       7090
strict                         6285
strict_14_with_grace_period      67
Name: cancellation_policy, dtype: int64

> I decided to categorize super_strict_30 and super_strict_60 into strict category.

In [43]:
df_list.cancellation_policy.replace({
    'super_strict_30':'strict_14_with_grace_period',
    'super_strict_60':'strict_14_with_grace_period',
    'strict_14_with_grace_period':'strict',
}, inplace=True)

In [44]:
df_list.cancellation_policy.value_counts()

flexible    9101
moderate    7090
strict      6352
Name: cancellation_policy, dtype: int64

### 2.10 Change Type of Categorical 

In [40]:
df_list.property_type = df_list.property_type.astype('category')
df_list.room_type = df_list.room_type.astype('category')
df_list.instant_bookable = df_list.instant_bookable.astype('category')
df_list.cancellation_policy = df_list.cancellation_policy.astype('category')

In [45]:
df_list.to_csv('clean_listing.csv', index=False)